# MultibodyPlant 랜더링하기 튜터리얼(Rendering MultibodyPlant Tutorial)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


여기서 다음과 같은 예제를 보여준다.:

* `MultibodyPlant`와 `SceneGraph`를 diagram에 추가하기
* 2개 별도 IIWA 로봇을 `MultibodyPlant`에 추가하기
* 기본 시각화 추가하기
* VTK 렌더러를 가지는 카메라 추가하기
* 색상 및 레이블 이미지 렌더링 (zero configuration)
* `SceneGraphInspector`를 사용하여 `SceneGraph` geometries에 대한 query
* `SceneGraph` geometries를 `MultibodyPlant` bodies와 연결
* 지정한 geometries에서 `RenderLabel` 추출
* 레이블을 다시 매핑하여 `ModelInstanceIndex`만 구별

## Necessary Imports

In [ ]:
import copy
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [ ]:
from pydrake.geometry import (
    ClippingRange,
    ColorRenderCamera,
    DepthRange,
    DepthRenderCamera,
    MakeRenderEngineVtk,
    RenderCameraCore,
    RenderEngineVtkParams,
    RenderLabel,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import BodyIndex
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.sensors import (
    CameraInfo,
    RgbdSensor,
)
from pydrake.visualization import (
    AddDefaultVisualization,
    ColorizeDepthImage,
    ColorizeLabelImage,
)

여기서 우리는 `Meshcat` 인스턴스를 시작할 것이다. Jupyter notebook 시작할때 한 번만 하고, 전체 notebook에서 동일한 인스턴스를 사용하는 것이 일반적이다. cell 출력에 표시되는 URL을 클릭하여 두 번째 브라우저 창을 열어야 한다.

In [ ]:
meshcat = StartMeshcat()

## helper 메소드 정의(Define helper methods)

In [ ]:
def xyz_rpy_deg(xyz, rpy_deg):
    """Shorthand for defining a pose."""
    rpy_deg = np.asarray(rpy_deg)
    return RigidTransform(RollPitchYaw(rpy_deg * np.pi / 180), xyz)

## plant와 scene graph를 가지는 diagram builder 생성하기(Create diagram builder with plant and scene graph.)

In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)

In [ ]:
iiwa_url = (
   "package://drake/manipulation/models/iiwa_description/sdf/"
   "iiwa14_no_collision.sdf"
)

## 왼쪽에 첫번째 IIWA 추가하기(Add first IIWA on left side.)

In [ ]:
(left_iiwa,) = Parser(plant, "left").AddModels(url=iiwa_url)
plant.WeldFrames(
    frame_on_parent_F=plant.world_frame(),
    frame_on_child_M=plant.GetFrameByName("iiwa_link_0", left_iiwa),
    X_FM=xyz_rpy_deg([0, -0.5, 0], [0, 0, 0]),
)

## 오른쪽에 2번째 IIWA 추가하기(Add second IIWA on right side.)

In [ ]:
(right_iiwa,) = Parser(plant, "right").AddModels(url=iiwa_url)
plant.WeldFrames(
    frame_on_parent_F=plant.world_frame(),
    frame_on_child_M=plant.GetFrameByName("iiwa_link_0", right_iiwa),
    X_FM=xyz_rpy_deg([0, 0.5, 0], [0, 0, 0]),
)

## free body 추가하기(Add a free body.)

In [ ]:
# Add a mesh from https://github.com/RobotLocomotion/models/ directly, without
# using the SDFormat wrapper file.
sugar_box_url = "package://drake_models/ycb/meshes/004_sugar_box_textured.obj"
(sugar_box,) = Parser(plant).AddModels(url=sugar_box_url)
sugar_box_body = plant.GetBodyByName("004_sugar_box_textured", sugar_box)
plant.SetDefaultFreeBodyPose(sugar_box_body, xyz_rpy_deg([0, 0, 0.5], [0, 0, 0]))

## 랜더러 추가하기(Add renderer.)

In [ ]:
renderer_name = "renderer"
scene_graph.AddRenderer(
    renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))

## 동일한 색상과 depth 속성을 갖는 카메라 추가하기(Add camera with same color and depth properties.)

In [ ]:
# N.B. These properties are chosen arbitrarily.
intrinsics = CameraInfo(
    width=640,
    height=480,
    fov_y=np.pi/4,
)
core = RenderCameraCore(
    renderer_name,
    intrinsics,
    ClippingRange(0.01, 10.0),
    RigidTransform(),
)
color_camera = ColorRenderCamera(core, show_window=False)
depth_camera = DepthRenderCamera(core, DepthRange(0.01, 10.0))

`plant.world_body()`에 카메라를 추가했기 때문에 이 카메라는 상황 고정(*scene-fixed*) 카메라이다.

카메라를 움직이게 하려면 world에 고정되지 않은 body에 카메라를 추가하기만 하면 된다.

In [ ]:
world_id = plant.GetBodyFrameIdOrThrow(plant.world_body().index())
X_WB = xyz_rpy_deg([2, 0, 0.75], [-90, 0, 90])
sensor = RgbdSensor(
    world_id,
    X_PB=X_WB,
    color_camera=color_camera,
    depth_camera=depth_camera,
)

builder.AddSystem(sensor)
builder.Connect(
    scene_graph.get_query_output_port(),
    sensor.query_object_input_port(),
)

## Add depth and label colorizers.
The systems convert a depth (or label) image into a color image, so we can easily preview it.

In [ ]:
colorize_depth = builder.AddSystem(ColorizeDepthImage())
colorize_label = builder.AddSystem(ColorizeLabelImage())
colorize_label.background_color.set([0,0,0])
builder.Connect(sensor.GetOutputPort("depth_image_32f"),
                colorize_depth.GetInputPort("depth_image_32f"))
builder.Connect(sensor.GetOutputPort("label_image"),
                colorize_label.GetInputPort("label_image"))

## Finalize the plant.

In [ ]:
plant.Finalize()

## 시각화 추가하기(Add visualization.)

`Meshcat`과 `DrakeVisualizer` 모두 기본 설정을 사용하여 시각화를 추가하는 것은 단일 함수 호출인 `AddDefaultVisualization()`으로 간단하게 수행할 수 있다.

In [ ]:
AddDefaultVisualization(builder=builder, meshcat=meshcat)

## diagram 만들기(Build the diagram.)

In [ ]:
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

## 기본 context로 초기 시각화 메시지를 publish하기(Publish initial visualization message with default context.)

In [ ]:
# TODO(eric.cousineau): Replace this with `diagram.Publish(diagram_context)`
# once all visualizers no longer use initialization events.
Simulator(diagram).Initialize()

## matplotlib를 사용하여 색상과 레이블 이미지를 랜더링하기(Render color and label images using matplotlib)
`body.index()`을 사용해서 기본 레이블 설정을 사용한다.

In [ ]:
color = sensor.color_image_output_port().Eval(
    sensor.GetMyContextFromRoot(diagram_context)).data
depth = colorize_depth.get_output_port().Eval(
    colorize_depth.GetMyContextFromRoot(diagram_context)).data
label = colorize_label.get_output_port().Eval(
    colorize_label.GetMyContextFromRoot(diagram_context)).data

fig, ax = plt.subplots(1, 3, figsize=(15, 10))
ax[0].imshow(color)
ax[1].imshow(depth)
ax[2].imshow(label)

## body index 대신에 레이블을 모델 인스턴스로 변경하기(Change labels to model instance instead of body index.)

각 geometry 아이템을 반복하여 레이블을 바디 인덱스 대신 모델 인스턴스를 반영하도록 변경할 것이다.

In [ ]:
source_id = plant.get_source_id()
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(diagram_context)
query_object = scene_graph.get_query_output_port().Eval(scene_graph_context)
inspector = query_object.inspector()
for geometry_id in inspector.GetAllGeometryIds():
    properties = copy.deepcopy(inspector.GetPerceptionProperties(geometry_id))
    if properties is None:
        continue
    frame_id = inspector.GetFrameId(geometry_id)
    body = plant.GetBodyFromFrameId(frame_id)
    new_label = int(body.model_instance())
    properties.UpdateProperty("label", "id", RenderLabel(new_label))
    # TODO(#19123) Ideally we would use AssignRole(..., kReplace) here,
    # but it is not yet supported for perception properties.
    scene_graph.RemoveRole(scene_graph_context, source_id, geometry_id, Role.kPerception)
    scene_graph.AssignRole(scene_graph_context, source_id, geometry_id, properties)

label = colorize_label.get_output_port().Eval(
    colorize_label.GetMyContextFromRoot(diagram_context)).data
plt.figure(figsize=(5, 5))
plt.imshow(label)